In [2]:
import optuna
from optuna import Trial

from math import sqrt
from typing import Tuple, List

import numpy as np
import pandas as pd
from mordred import Calculator, descriptors
#import openbabel
from openbabel import pybel
from PyBioMed.PyMolecule.fingerprint import CalculatePubChemFingerprint,CalculateECFP2Fingerprint
from rdkit import Chem
from rdkit.Chem.rdchem import Atom

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import StratifiedKFold, KFold


from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as G_Loader 
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import BatchNorm


# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
# Pytorch and Pytorch Geometric
import torch

import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
import torch.nn.functional as F # activation function
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as V_Loader # dataset management
%run ./graph_feature.ipynb 
%run ./dataset_processing.ipynb 

%run ./my_performances.ipynb 


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_idx = np.load('train_indices.npy')
val_idx = np.load('val_indices.npy')
test_idx = np.load('test_indices.npy')

In [4]:
# load structured graph data
k=10
data_list_train=[]
for i in range(k):
    one_fold= []
    for idx in range(train_idx[i]):
        one_graph_data =torch.load(f"data_train_{i}/tensor{idx}.pt")
        one_fold.append(one_graph_data)
    data_list_train.append(one_fold)
    
data_list_val=[]
for i in range(k):
    one_fold= []
    for idx in range(val_idx[i]):
        one_graph_data =torch.load(f"data_val_{i}/tensor{idx}.pt")
        one_fold.append(one_graph_data)
    data_list_val.append(one_fold)
    
data_list_test =[]
for idx in range(test_idx):
    one_graph_data =torch.load(f"data_test/tensor{idx}.pt")
    data_list_test.append(one_graph_data)
    
# load the output label 
total_train_targets =[]
total_validation_targets =[]
total_test_targets=[]
for i in range(k):
    total_train_targets.append(np.load('total_train_targets'+ str(i)+'.npy'))
    total_validation_targets.append(np.load('total_validation_targets' +str(i)+'.npy'))

total_test_targets= np.load('total_test_targets.npy')

In [5]:
# create dataloader for training (graph data)
#======================================================================================
list_data_graph_train =[]
list_data_graph_val =[]
for data_train, data_val in zip(data_list_train, data_list_val):
    train_loader = G_Loader(dataset = data_train, batch_size = 126)
    val_loader = G_Loader(dataset = data_val, batch_size = 126)
    list_data_graph_train.append(train_loader)
    list_data_graph_val.append(val_loader)

# create dataloader for test (graph data)
#======================================================================================
g_test_loader = G_Loader(dataset = data_list_test, batch_size = 1)

In [6]:
# criterion for loss function
#=======================================================================================
criterion = torch.nn.BCELoss()

#optimization method 
#=======================================================================================
def get_optimizer(gnn_model, learning_rate, optimizer_type, decay):
    if optimizer_type==1:
        optimizer = torch.optim.SGD(gnn_model.parameters(), lr=learning_rate, momentum=0.9)
    if optimizer_type==2:
        optimizer = torch.optim.Adam(gnn_model.parameters(), lr=learning_rate, weight_decay =decay)
    if optimizer_type ==3 :
        optimizer = torch.optim.Adamax(gnn_model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
        
    return optimizer

In [7]:
class modelA(torch.nn.Module):
    def __init__(self, hidden_channels1,hidden_channels2, num_node_features,heads1,heads2
                 ,dropout_rateA,dropout_rateB, dropout_rateC,dense_layer1):
        super(modelA, self).__init__()
        
        torch.manual_seed(12345)
        self.conv1 = GATConv(num_node_features, hidden_channels1,heads1)
        self.conv2 = GATConv(hidden_channels1*heads1,hidden_channels2, heads2)
        
        self.bn1 = BatchNorm (hidden_channels1*heads1)
        self.bn2 = BatchNorm (hidden_channels2*heads2)
        
        self.dropoutA = dropout_rateA
        self.dropoutB = dropout_rateB
        self.dropoutC = dropout_rateC
        
        self.lin1 = Linear(hidden_channels2*heads2,dense_layer1)
        #self.lin1 = Linear(hidden_channels2*heads2,dense_layer1)
        self.lin2=  Linear(dense_layer1,1)
        
    def forward(self, x, edge_index, batch):

        x = self.conv1(x, edge_index)
        
        x = F.dropout(x, p=self.dropoutA , training=self.training)
        x = x.relu()
        x = self.bn1(x)
        x = self.conv2(x, edge_index)
        
        x = F.dropout(x, p=self.dropoutB , training=self.training)
        x = x.relu()
        x = self.bn2(x)  
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        
        x = self.lin1(x)
        x = F.dropout(x, p=self.dropoutC , training=self.training)
        x = x.relu()
        x = self.lin2(x)
        return torch.sigmoid(x)


In [8]:
def train_1(train_g_loader, combined_model, optimizer):
    combined_model.train()
    
    for data_X1 in train_g_loader:  # Iterate in batches over the training dataset.
        out = combined_model(data_X1.x, data_X1.edge_index, data_X1.batch)  # Perform a single forward pass.
        
        y_t = data_X1.y.type(torch.FloatTensor)
        loss = criterion(out[:,0], y_t)  # Compute the loss.
        #print(k)
        #print(loss)
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return loss, combined_model, optimizer

def validation_1(val_g_loader, combined_model):
    for data_X1 in val_g_loader: # Iterate in batches over the training dataset.
        out = combined_model(data_X1.x, data_X1.edge_index, data_X1.batch)  # Perform a single forward pass.
        y_t = data_X1.y.type(torch.FloatTensor)
        val_loss = criterion(out[:,0], y_t)  # Compute the loss.
        
    return val_loss

In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

# performances visualization 
import matplotlib.pyplot as plt
#import seaborn as sns
import statistics
from prettytable import PrettyTable
%run ./my_performances.ipynb 


def test_1(g_loader, combined_model):
    combined_model.eval()
    list_pred =[]
    list_targets =[]
    correct = 0
    for data_X1 in g_loader:  # Iterate in batches over the training/test dataset.
            out = combined_model(data_X1.x, data_X1.edge_index, data_X1.batch)
            out_1 = out[:,0]
            
            list_pred.append(out_1.item())
            list_targets.append(data_X1.y.item())
    return list_pred, list_targets 


def test_2(g_loader, combined_model):
    combined_model.eval()
    list_pred =[]
    list_targets =[]
    correct = 0
    for data_X1 in g_loader:  # Iterate in batches over the training/test dataset.
            out = combined_model(data_X1.x, data_X1.edge_index, data_X1.batch)
            out_1 = out[:,0]
            
            list_pred = list_pred + out_1.tolist()
            list_targets= list_targets + data_X1.y.tolist()
    return list_pred, list_targets 
# used to count the train accuracy ,and validation accuracy when in the training mode 
def test(g_loader, combined_model):
    combined_model.eval()

    correct = 0
    for data_X1 in g_loader:  # Iterate in batches over the training/test dataset.
            out = combined_model(data_X1.x, data_X1.edge_index, data_X1.batch)
            out_1 = out[:,0]
            for i,value in enumerate(out_1) :
                if value > 0.5 :
                    out_1[i] = 1
                else : out_1[i] = 0
            pred = out_1  # Use the class with highest probability.
            correct += int((pred == data_X1.y).sum())  # Check against ground-truth labels.
    return correct / len(g_loader.dataset)  # Derive ratio of correct predictions.

In [ ]:
parameters: {'hidden_channels1 ': 100, 'hidden_channels2 ': 112, 'heads1': 10, 'heads2': 10, 'dropout_rateA': 0.40302653363220353, 'dropout_rateB': 0.2619322952835395, 'dropout_rateC': 0.3233230921356046, 'dense_layer1': 86, 'dense_layer2': 39, 'dense_layer3': 16, 'weight_decay': 0.0001959090324617911, 'learning_rate': 0.00015144009883033432}

In [17]:


hidden_channels1 =82
hidden_channels2 =82
num_node_features=79
heads1 =9
heads2 =9
dropout_rateA=0.20184714781996305
dropout_rateB=0.26157496401430025
dropout_rateC=0.12826012269932247
dense_layer1 =64

learning_rate = 0.0015061165279742287
optimizer_type = 2
weight_decay=0.00017722962529205202



total_history_train_loss=[]
total_history_val_loss=[]
total_history_train_acc=[]
total_history_val_acc=[]
list_trained_model =[]

list_trained_model =[]
list_val_acc=[]
k=10
for i in range(k):
    combined_model          = modelA(hidden_channels1,hidden_channels2, num_node_features,heads1,heads2
                 ,dropout_rateA,dropout_rateB, dropout_rateC,dense_layer1)
    optimizer               = get_optimizer(combined_model, learning_rate, optimizer_type,weight_decay)
   
    history_train_loss=[]
    history_val_loss=[]
    history_train_acc=[]
    history_val_acc=[]
        
    for epoch in range(1,50):            
        train_loss,combined_model, optimizer = train_1(list_data_graph_train[i],combined_model,optimizer)
        val_loss                             = validation_1(list_data_graph_val[i],combined_model) 
        
        train_acc                            = test(list_data_graph_train[i],combined_model)
        val_acc                              = test(list_data_graph_val[i],combined_model)
        
           
            
        history_train_loss.append(train_loss.item())
        history_val_loss.append(val_loss.item())
        history_train_acc.append(train_acc)
        history_val_acc.append(val_acc)
        
        print(f'Fold: {i}')
        print(f'Epoch: {epoch:03d}, Train Loss:, {train_loss.item():.4f}')
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
        print(f'Epoch: {epoch:03d}, val Loss:, {val_loss.item():.4f}')
        print(f'Epoch: {epoch:03d}, val Acc: {val_acc:.4f}')
        print(" ")
    
    
    total_history_train_loss.append(history_train_loss)
    total_history_val_loss.append(history_val_loss)
    total_history_train_acc.append(history_train_acc)
    total_history_val_acc.append(history_val_acc)
    list_trained_model.append(combined_model)

Fold: 0
Epoch: 001, Train Loss:, 0.5600
Epoch: 001, Train Acc: 0.6670
Epoch: 001, val Loss:, 0.5841
Epoch: 001, val Acc: 0.6667
 
Fold: 0
Epoch: 002, Train Loss:, 0.5336
Epoch: 002, Train Acc: 0.6670
Epoch: 002, val Loss:, 0.5659
Epoch: 002, val Acc: 0.6667
 
Fold: 0
Epoch: 003, Train Loss:, 0.4921
Epoch: 003, Train Acc: 0.6696
Epoch: 003, val Loss:, 0.5310
Epoch: 003, val Acc: 0.6667
 
Fold: 0
Epoch: 004, Train Loss:, 0.4758
Epoch: 004, Train Acc: 0.6890
Epoch: 004, val Loss:, 0.5271
Epoch: 004, val Acc: 0.7063
 
Fold: 0
Epoch: 005, Train Loss:, 0.4304
Epoch: 005, Train Acc: 0.7164
Epoch: 005, val Loss:, 0.5403
Epoch: 005, val Acc: 0.7381
 
Fold: 0
Epoch: 006, Train Loss:, 0.4333
Epoch: 006, Train Acc: 0.7465
Epoch: 006, val Loss:, 0.5401
Epoch: 006, val Acc: 0.7619
 
Fold: 0
Epoch: 007, Train Loss:, 0.4060
Epoch: 007, Train Acc: 0.7518
Epoch: 007, val Loss:, 0.5434
Epoch: 007, val Acc: 0.7857
 
Fold: 0
Epoch: 008, Train Loss:, 0.3712
Epoch: 008, Train Acc: 0.7818
Epoch: 008, val Loss

Fold: 1
Epoch: 016, Train Loss:, 0.2573
Epoch: 016, Train Acc: 0.8587
Epoch: 016, val Loss:, 0.6588
Epoch: 016, val Acc: 0.6984
 
Fold: 1
Epoch: 017, Train Loss:, 0.2344
Epoch: 017, Train Acc: 0.8436
Epoch: 017, val Loss:, 0.6983
Epoch: 017, val Acc: 0.6984
 
Fold: 1
Epoch: 018, Train Loss:, 0.2908
Epoch: 018, Train Acc: 0.8410
Epoch: 018, val Loss:, 0.7267
Epoch: 018, val Acc: 0.6984
 
Fold: 1
Epoch: 019, Train Loss:, 0.2231
Epoch: 019, Train Acc: 0.8719
Epoch: 019, val Loss:, 0.8045
Epoch: 019, val Acc: 0.6508
 
Fold: 1
Epoch: 020, Train Loss:, 0.2681
Epoch: 020, Train Acc: 0.8648
Epoch: 020, val Loss:, 0.6884
Epoch: 020, val Acc: 0.7540
 
Fold: 1
Epoch: 021, Train Loss:, 0.2984
Epoch: 021, Train Acc: 0.8463
Epoch: 021, val Loss:, 0.8799
Epoch: 021, val Acc: 0.7063
 
Fold: 1
Epoch: 022, Train Loss:, 0.2991
Epoch: 022, Train Acc: 0.8754
Epoch: 022, val Loss:, 0.7910
Epoch: 022, val Acc: 0.6905
 
Fold: 1
Epoch: 023, Train Loss:, 0.2454
Epoch: 023, Train Acc: 0.8746
Epoch: 023, val Loss

Fold: 2
Epoch: 031, Train Loss:, 0.1187
Epoch: 031, Train Acc: 0.9267
Epoch: 031, val Loss:, 0.9813
Epoch: 031, val Acc: 0.6429
 
Fold: 2
Epoch: 032, Train Loss:, 0.1237
Epoch: 032, Train Acc: 0.9258
Epoch: 032, val Loss:, 1.0440
Epoch: 032, val Acc: 0.6349
 
Fold: 2
Epoch: 033, Train Loss:, 0.1278
Epoch: 033, Train Acc: 0.9152
Epoch: 033, val Loss:, 1.2887
Epoch: 033, val Acc: 0.5556
 
Fold: 2
Epoch: 034, Train Loss:, 0.0867
Epoch: 034, Train Acc: 0.9037
Epoch: 034, val Loss:, 0.9815
Epoch: 034, val Acc: 0.6270
 
Fold: 2
Epoch: 035, Train Loss:, 0.1282
Epoch: 035, Train Acc: 0.9258
Epoch: 035, val Loss:, 0.9047
Epoch: 035, val Acc: 0.6190
 
Fold: 2
Epoch: 036, Train Loss:, 0.1726
Epoch: 036, Train Acc: 0.9223
Epoch: 036, val Loss:, 1.2066
Epoch: 036, val Acc: 0.6190
 
Fold: 2
Epoch: 037, Train Loss:, 0.1121
Epoch: 037, Train Acc: 0.9258
Epoch: 037, val Loss:, 1.3302
Epoch: 037, val Acc: 0.6111
 
Fold: 2
Epoch: 038, Train Loss:, 0.1744
Epoch: 038, Train Acc: 0.9258
Epoch: 038, val Loss

Fold: 3
Epoch: 046, Train Loss:, 0.0902
Epoch: 046, Train Acc: 0.9311
Epoch: 046, val Loss:, 1.5569
Epoch: 046, val Acc: 0.6429
 
Fold: 3
Epoch: 047, Train Loss:, 0.0590
Epoch: 047, Train Acc: 0.9196
Epoch: 047, val Loss:, 1.4362
Epoch: 047, val Acc: 0.6587
 
Fold: 3
Epoch: 048, Train Loss:, 0.0824
Epoch: 048, Train Acc: 0.9249
Epoch: 048, val Loss:, 1.7414
Epoch: 048, val Acc: 0.6587
 
Fold: 3
Epoch: 049, Train Loss:, 0.0781
Epoch: 049, Train Acc: 0.9408
Epoch: 049, val Loss:, 1.5654
Epoch: 049, val Acc: 0.6429
 
Fold: 4
Epoch: 001, Train Loss:, 0.5494
Epoch: 001, Train Acc: 0.6670
Epoch: 001, val Loss:, 0.6792
Epoch: 001, val Acc: 0.6667
 
Fold: 4
Epoch: 002, Train Loss:, 0.5314
Epoch: 002, Train Acc: 0.6696
Epoch: 002, val Loss:, 0.6751
Epoch: 002, val Acc: 0.6825
 
Fold: 4
Epoch: 003, Train Loss:, 0.4935
Epoch: 003, Train Acc: 0.6864
Epoch: 003, val Loss:, 0.7090
Epoch: 003, val Acc: 0.6270
 
Fold: 4
Epoch: 004, Train Loss:, 0.4658
Epoch: 004, Train Acc: 0.7067
Epoch: 004, val Loss

Fold: 5
Epoch: 012, Train Loss:, 0.3301
Epoch: 012, Train Acc: 0.8525
Epoch: 012, val Loss:, 0.7452
Epoch: 012, val Acc: 0.6984
 
Fold: 5
Epoch: 013, Train Loss:, 0.2905
Epoch: 013, Train Acc: 0.8657
Epoch: 013, val Loss:, 0.8030
Epoch: 013, val Acc: 0.6667
 
Fold: 5
Epoch: 014, Train Loss:, 0.2800
Epoch: 014, Train Acc: 0.8534
Epoch: 014, val Loss:, 0.8513
Epoch: 014, val Acc: 0.6746
 
Fold: 5
Epoch: 015, Train Loss:, 0.2870
Epoch: 015, Train Acc: 0.8542
Epoch: 015, val Loss:, 0.8377
Epoch: 015, val Acc: 0.6587
 
Fold: 5
Epoch: 016, Train Loss:, 0.3162
Epoch: 016, Train Acc: 0.8419
Epoch: 016, val Loss:, 0.8628
Epoch: 016, val Acc: 0.6905
 
Fold: 5
Epoch: 017, Train Loss:, 0.2763
Epoch: 017, Train Acc: 0.8401
Epoch: 017, val Loss:, 0.8941
Epoch: 017, val Acc: 0.6825
 
Fold: 5
Epoch: 018, Train Loss:, 0.3334
Epoch: 018, Train Acc: 0.8595
Epoch: 018, val Loss:, 0.8186
Epoch: 018, val Acc: 0.6508
 
Fold: 5
Epoch: 019, Train Loss:, 0.2446
Epoch: 019, Train Acc: 0.8807
Epoch: 019, val Loss

Fold: 6
Epoch: 027, Train Loss:, 0.2059
Epoch: 027, Train Acc: 0.8843
Epoch: 027, val Loss:, 1.0917
Epoch: 027, val Acc: 0.6429
 
Fold: 6
Epoch: 028, Train Loss:, 0.1476
Epoch: 028, Train Acc: 0.8799
Epoch: 028, val Loss:, 1.0140
Epoch: 028, val Acc: 0.6508
 
Fold: 6
Epoch: 029, Train Loss:, 0.1328
Epoch: 029, Train Acc: 0.8993
Epoch: 029, val Loss:, 1.0254
Epoch: 029, val Acc: 0.6032
 
Fold: 6
Epoch: 030, Train Loss:, 0.1870
Epoch: 030, Train Acc: 0.8949
Epoch: 030, val Loss:, 1.1098
Epoch: 030, val Acc: 0.6508
 
Fold: 6
Epoch: 031, Train Loss:, 0.1535
Epoch: 031, Train Acc: 0.9170
Epoch: 031, val Loss:, 1.1529
Epoch: 031, val Acc: 0.6429
 
Fold: 6
Epoch: 032, Train Loss:, 0.1497
Epoch: 032, Train Acc: 0.9214
Epoch: 032, val Loss:, 1.0851
Epoch: 032, val Acc: 0.6429
 
Fold: 6
Epoch: 033, Train Loss:, 0.1409
Epoch: 033, Train Acc: 0.9108
Epoch: 033, val Loss:, 1.1927
Epoch: 033, val Acc: 0.6429
 
Fold: 6
Epoch: 034, Train Loss:, 0.1854
Epoch: 034, Train Acc: 0.9081
Epoch: 034, val Loss

Fold: 7
Epoch: 042, Train Loss:, 0.0722
Epoch: 042, Train Acc: 0.9302
Epoch: 042, val Loss:, 1.9112
Epoch: 042, val Acc: 0.6349
 
Fold: 7
Epoch: 043, Train Loss:, 0.1427
Epoch: 043, Train Acc: 0.9329
Epoch: 043, val Loss:, 1.8732
Epoch: 043, val Acc: 0.7143
 
Fold: 7
Epoch: 044, Train Loss:, 0.0943
Epoch: 044, Train Acc: 0.9408
Epoch: 044, val Loss:, 1.8726
Epoch: 044, val Acc: 0.6905
 
Fold: 7
Epoch: 045, Train Loss:, 0.1248
Epoch: 045, Train Acc: 0.9479
Epoch: 045, val Loss:, 1.7775
Epoch: 045, val Acc: 0.6508
 
Fold: 7
Epoch: 046, Train Loss:, 0.0661
Epoch: 046, Train Acc: 0.9452
Epoch: 046, val Loss:, 1.8353
Epoch: 046, val Acc: 0.6270
 
Fold: 7
Epoch: 047, Train Loss:, 0.0818
Epoch: 047, Train Acc: 0.9461
Epoch: 047, val Loss:, 1.6902
Epoch: 047, val Acc: 0.7063
 
Fold: 7
Epoch: 048, Train Loss:, 0.0667
Epoch: 048, Train Acc: 0.9337
Epoch: 048, val Loss:, 1.8630
Epoch: 048, val Acc: 0.6587
 
Fold: 7
Epoch: 049, Train Loss:, 0.1027
Epoch: 049, Train Acc: 0.9329
Epoch: 049, val Loss

Fold: 9
Epoch: 008, Train Loss:, 0.4219
Epoch: 008, Train Acc: 0.7679
Epoch: 008, val Loss:, 0.6047
Epoch: 008, val Acc: 0.7040
 
Fold: 9
Epoch: 009, Train Loss:, 0.4021
Epoch: 009, Train Acc: 0.8085
Epoch: 009, val Loss:, 0.6310
Epoch: 009, val Acc: 0.6880
 
Fold: 9
Epoch: 010, Train Loss:, 0.3849
Epoch: 010, Train Acc: 0.8173
Epoch: 010, val Loss:, 0.6456
Epoch: 010, val Acc: 0.6880
 
Fold: 9
Epoch: 011, Train Loss:, 0.3944
Epoch: 011, Train Acc: 0.8350
Epoch: 011, val Loss:, 0.6992
Epoch: 011, val Acc: 0.7120
 
Fold: 9
Epoch: 012, Train Loss:, 0.3620
Epoch: 012, Train Acc: 0.8085
Epoch: 012, val Loss:, 0.6855
Epoch: 012, val Acc: 0.7040
 
Fold: 9
Epoch: 013, Train Loss:, 0.3290
Epoch: 013, Train Acc: 0.8491
Epoch: 013, val Loss:, 0.6742
Epoch: 013, val Acc: 0.6880
 
Fold: 9
Epoch: 014, Train Loss:, 0.3288
Epoch: 014, Train Acc: 0.8658
Epoch: 014, val Loss:, 0.7396
Epoch: 014, val Acc: 0.6720
 
Fold: 9
Epoch: 015, Train Loss:, 0.2940
Epoch: 015, Train Acc: 0.8473
Epoch: 015, val Loss

In [18]:
# RUNNING THE 10 PRETRAINED MODEL USING INDEPENDENT DATA
#======================================================================
nCV= 10 # ten crossfold validation 
list_fold_pred =[]
list_fold_targets =[]


g_test_loader = G_Loader(dataset = data_list_test, batch_size = 1)

for combined_model in list_trained_model:  
    list_pred, list_targets = test_1(g_test_loader,combined_model)
    list_fold_pred.append(list_pred)
    list_fold_targets.append(list_targets)

In [19]:
# GET THE PERFORMANCES FROM THE TEST
#========================================================================
total_performances = performances(list_fold_pred, list_fold_targets, nCV)
list_bal_acc = []
for sen, spec in zip (total_performances[1] , total_performances[2]):
    bal_acc = (sen + spec)/2
    list_bal_acc.append(bal_acc)
                
statistics.mean(list_bal_acc)

0.649125

In [20]:
import statistics
from prettytable import PrettyTable
perf = total_performances
model_title = 'Test Perf FCNN'
data_type ='Fusion'
Create_Tables(perf, model_title, data_type)

+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type | m_ACC |  m_SN |  m_SP | m_MCC | m_AUC | m_Kappa | m_AUROC | m_Bal_ACC |  m_F1 |
+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN |   Fusion  | 0.718 | 0.831 | 0.467 | 0.314 | 0.683 |  0.312  |  0.683  |   0.649   | 0.802 |
+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
+----------------+-----------+-------+-------+------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type | e_ACC |  e_SN |  SP  | e_MCC | e_AUC | e_Kappa | e_AUROC | e_Bal_ACC |  e_F1 |
+----------------+-----------+-------+-------+------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN |   Fusion  |  0.01 | 0.019 | 0.03 | 0.023 | 0.019 |  0.023  |  0.019  |   0.012   | 0.802 |
+----